This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5rc3) and depends on Magma (tested using version
2.26-9).

In this notebook, we make an intermediate calculation towards finding generic curves of genus 6 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1.  Allow 10 minutes for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $G(2,5)$, represented as $2 \times 5$ echelon matrices,
together with their Plucker coordinates.

In [2]:
F = GF(2)

In [3]:
coords = {}
for V in VectorSpace(F, 5).subspaces(2):
    M = V.matrix()
    M.set_immutable()
    coords[M] = vector(M.minors(2))
S = list(coords.keys())
len(S)

155

Construct the Plucker relations defining $G(2,5)$ inside $\mathbb{P}^9$, then verify that these relations vanish on the points we have constructed.

In [4]:
P.<x01,x02,x03,x04,x12,x13,x14,x23,x24,x34> = PolynomialRing(F, 10)
quads = (x01*x23 + x02*x13 + x03*x12,
         x01*x24 + x02*x14 + x04*x12,
         x01*x34 + x03*x14 + x04*x13,
         x02*x34 + x03*x24 + x04*x23,
         x12*x34 + x13*x24 + x14*x23)
assert all(f(*v) == 0 for f in quads for v in coords.values())

Construct an orbit lookup tree of depth 6. We forbid 5-tuples whose linear span is only 3-dimensional, as these can only occur for plane quintics.

In [5]:
def vecs_to_gens(vecs):
    return tuple(sum(P.gens()[i] * v[i] for i in range(10)) for v in vecs)

In [6]:
def linear_section(coords):
    V = Matrix(coords).right_kernel()
    tmp2 = vecs_to_gens(V.gens()) + quads
    return P.ideal(tmp2)

In [7]:
def apply_group_elem(g, M):
    M1 = (M*~g).echelon_form()
    M1.set_immutable()
    return M1

In [8]:
def stabilizer(M):
    # First construct the stabilizer of the span of e1, e2.
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in GL(2, F).gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in GL(3, F).gens()]
    l0.append(identity_matrix(5))
    l0[-1][3,0] = 1
    # Construct a matrix to conjugate these generators.
    l = M.rows()
    for i in range(5):
        if i not in M.pivots():
            l.append(vector(1 if j==i else 0 for j in range(5)))
    M1 = Matrix(F, l)
    G = GL(5, F).subgroup([~M1*g*M1 for g in l0])
    assert all((M*g.matrix()).echelon_form() == M for g in G.gens())
    return G

In [9]:
def optimized_rep(g):
    return g.matrix()

In [10]:
def forbid(mats, easy=False, coords=coords):
    tmp = [coords[M] for M in mats]
    if len(mats) == 5 and Matrix(tmp).rank() <= 3:
        return True
    J = linear_section(tmp)
    if J.dimension() > 3:
        return True
    if easy:
        return False
    return False

In [11]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': vec_stab,
           'optimized_rep': optimized_rep,
           'forbid': forbid}

In [12]:
G = GL(5, F)
tree = build_orbit_tree(G, S, 6, methods, verbose=False)

In [13]:
[len(green_nodes(tree, i)) for i in range(4,7)]

[39, 235, 2201]

Write coordinates of representative tuples to a file.

In [14]:
l = [[coords[M] for M in mats] for i in range(4, 7) for mats in green_nodes(tree, i)]
with open("genus6-tuples.txt", "w") as f:
    f.write(str(l))

In [15]:
print("This stage complete!")

This stage complete!
